<a href="https://colab.research.google.com/github/NguyenVy2001/csv-file/blob/main/Project_Python_Sales_change_Revenue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import io
prices_csv = '/content/drive/MyDrive/Project Python CSV/prices.csv'
prices = pd.read_csv(prices_csv, sep = ';')
sales_csv = '/content/drive/MyDrive/Project Python CSV/sales.csv'
sales = pd.read_csv(sales_csv, sep = ';')
print(prices)
print(sales)

    product_id  old_price  new_price        updated_at
0           64     270000     239000  09/10/2018 16:37
1      3954203      60000      64000  09/11/2018 11:54
2      3954203      60500      57500     9/17/18 22:59
3      3954203      64000      60500      9/15/18 3:49
4      3954203      68800      60000  09/10/2018 16:32
5      3998909      15500      16500      9/16/18 5:09
6      3998909      17000      15500      9/13/18 6:43
7      3998909      19000      17000  09/10/2018 16:35
8      4085861      53500      52000     9/17/18 22:59
9      4085861      53500      67000   09/12/2018 3:51
10     4085861      58000      53500      9/17/18 3:35
11     4085861      60000      53500   09/11/2018 8:51
12     4085861      62500      58000      9/15/18 3:51
13     4085861      67000      62500      9/13/18 6:43
     product_id        ordered_at  quantity_ordered
0       3998909     9/18/18 17:51                 1
1       3998909     9/18/18 12:52                 1
2       3998909    

In [ ]:
sales_df = prices.merge(sales, on = 'product_id')
sales_df['ordered_at']= pd.to_datetime(sales_df['ordered_at'])
sales_df['updated_at']= pd.to_datetime(sales_df['updated_at'])
sales_df = sales_df.sort_values('product_id')
sales_df['time_diff'] = abs(sales_df['ordered_at'] - sales_df['updated_at'])
print(sales_df)

     product_id  old_price  new_price          updated_at          ordered_at  \
0            64     270000     239000 2018-09-10 16:37:00 2018-09-11 19:32:00   
1            64     270000     239000 2018-09-10 16:37:00 2018-09-14 20:10:00   
2            64     270000     239000 2018-09-10 16:37:00 2018-09-12 16:00:00   
3            64     270000     239000 2018-09-10 16:37:00 2018-09-12 13:23:00   
34      3954203      64000      60500 2018-09-15 03:49:00 2018-09-11 18:26:00   
..          ...        ...        ...                 ...                 ...   
393     4085861      58000      53500 2018-09-17 03:35:00 2018-09-16 11:23:00   
394     4085861      58000      53500 2018-09-17 03:35:00 2018-09-15 11:59:00   
395     4085861      58000      53500 2018-09-17 03:35:00 2018-09-16 18:35:00   
371     4085861      53500      67000 2018-09-12 03:51:00 2018-09-14 09:19:00   
953     4085861      67000      62500 2018-09-13 06:43:00 2018-09-13 23:29:00   

     quantity_ordered      

In [ ]:
min_time_diff= sales_df.groupby(['product_id', 'ordered_at'])['time_diff'].min().reset_index()
min_time_diff = min_time_diff.sort_values('product_id')
print(min_time_diff)

     product_id          ordered_at       time_diff
0            64 2018-09-11 19:32:00 1 days 02:55:00
1            64 2018-09-12 13:23:00 1 days 20:46:00
2            64 2018-09-12 16:00:00 1 days 23:23:00
3            64 2018-09-14 20:10:00 4 days 03:33:00
16      3954203 2018-09-18 21:26:00 0 days 22:27:00
..          ...                 ...             ...
79      4085861 2018-09-13 21:08:00 0 days 14:25:00
80      4085861 2018-09-13 22:05:00 0 days 15:22:00
81      4085861 2018-09-13 23:29:00 0 days 16:46:00
75      4085861 2018-09-13 19:01:00 0 days 12:18:00
172     4085861 2018-09-18 20:54:00 0 days 21:55:00

[173 rows x 3 columns]


In [ ]:
result = min_time_diff.merge(sales_df, on = ['time_diff', 'product_id', 'ordered_at'])
print(result)

     product_id          ordered_at       time_diff  old_price  new_price  \
0            64 2018-09-11 19:32:00 1 days 02:55:00     270000     239000   
1            64 2018-09-12 13:23:00 1 days 20:46:00     270000     239000   
2            64 2018-09-12 16:00:00 1 days 23:23:00     270000     239000   
3            64 2018-09-14 20:10:00 4 days 03:33:00     270000     239000   
4       3954203 2018-09-18 21:26:00 0 days 22:27:00      60500      57500   
..          ...                 ...             ...        ...        ...   
170     4085861 2018-09-13 21:08:00 0 days 14:25:00      67000      62500   
171     4085861 2018-09-13 22:05:00 0 days 15:22:00      67000      62500   
172     4085861 2018-09-13 23:29:00 0 days 16:46:00      67000      62500   
173     4085861 2018-09-13 19:01:00 0 days 12:18:00      67000      62500   
174     4085861 2018-09-18 20:54:00 0 days 21:55:00      53500      52000   

             updated_at  quantity_ordered  
0   2018-09-10 16:37:00        

In [ ]:
def calculate_final_price(x):
  if x['ordered_at'] > x['updated_at']:
    return x['new_price']
  else:
    return x['old_price']

result['final_price'] = result.apply(calculate_final_price, axis=1)


print(result)


     product_id          ordered_at       time_diff  old_price  new_price  \
0            64 2018-09-11 19:32:00 1 days 02:55:00     270000     239000   
1            64 2018-09-12 13:23:00 1 days 20:46:00     270000     239000   
2            64 2018-09-12 16:00:00 1 days 23:23:00     270000     239000   
3            64 2018-09-14 20:10:00 4 days 03:33:00     270000     239000   
4       3954203 2018-09-18 21:26:00 0 days 22:27:00      60500      57500   
..          ...                 ...             ...        ...        ...   
170     4085861 2018-09-13 21:08:00 0 days 14:25:00      67000      62500   
171     4085861 2018-09-13 22:05:00 0 days 15:22:00      67000      62500   
172     4085861 2018-09-13 23:29:00 0 days 16:46:00      67000      62500   
173     4085861 2018-09-13 19:01:00 0 days 12:18:00      67000      62500   
174     4085861 2018-09-18 20:54:00 0 days 21:55:00      53500      52000   

             updated_at  quantity_ordered  final_price  
0   2018-09-10 16:

In [ ]:
selected_df = result[['product_id', 'final_price', 'quantity_ordered']]
selected_df['revenue'] = selected_df['final_price'] * selected_df['quantity_ordered']
revenue_by_price = selected_df.groupby(['product_id','final_price'])['revenue'].sum().reset_index()
revenue_by_price = revenue_by_price.sort_values('product_id')
print(revenue_by_price)

    product_id  final_price  revenue
0           64       239000   956000
1      3954203        57500    57500
2      3954203        60000   180000
3      3954203        64000   640000
4      3998909        15500    15500
5      3998909        16500   231000
6      3998909        17000    34000
7      4085861        52000  1040000
8      4085861        53500  2140000
9      4085861        58000  2204000
10     4085861        60000   180000
11     4085861        62500  1812500
12     4085861        67000   871000


<ipython-input-9-131c0de2a6a6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['revenue'] = selected_df['final_price'] * selected_df['quantity_ordered']


In [ ]:
revenue_by_product = selected_df.groupby('product_id')['revenue'].sum().reset_index()
print(revenue_by_product)

   product_id  revenue
0          64   956000
1     3954203   877500
2     3998909   280500
3     4085861  8247500
